In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

sns.set(rc={'figure.figsize':(15,8)})

path_to_catalog=r'C:\Users\kgrzebien\Desktop\python_tasks\reklamacje\raport_felery\\'


In [39]:
def func_read_data(path):
    df=pd.read_csv(path+'.csv',
              encoding='Cp1250',
              sep="|")
    return df
    
def func_str_to_float(s):
    s=s.astype(str)
    s=s.apply(lambda x: x.replace(',','.'))
    s=s.astype(float)
    return s

def func_add_time(df):
    
    df['Z_DNIA']=pd.to_datetime(df['Z_DNIA'], format='%y/%m/%d')
    
    df['year']=df['Z_DNIA'].apply(lambda x: x.year)
    df['month']=df['Z_DNIA'].apply(lambda x: x.month)
    
    df=df.loc[df['Z_DNIA']<=np.datetime64('2021-05-13')]
    
    # df['day_of_year']=df['Z_DNIA'].apply(lambda x: x.timetuple().tm_yday)
    # df['week']=df['Z_DNIA'].apply(lambda x: x.isocalendar()[1])
    # df['week_day']=df['Z_DNIA'].apply(lambda x: x.isocalendar()[2])
    # df['year_month']=df['year'].astype(str)+'-'+df['month'].astype(str)+'-'+'01'
    # df['year_month']=df['year_month'].apply(lambda x: x[:5]+'0'+x[5:] if len(x)==9 else x)
    # df['year_month']=pd.to_datetime(df['year_month'], format='%Y-%m-%d')
    # df['year_week']=df['year'].astype(str)+'--'+df['week'].astype(str)
    # df['year_week']=df['year_week'].apply(lambda x: x[:6]+'0'+x[6:] if len(x)==7 else x)
    # df['year_day']=df['year'].astype(str)+'--'+df['day_of_year'].astype(str)
    # df['year_day']=df['year_day'].apply(lambda x: x[:6]+'0'+x[6:] if len(x)<9 else x)
    return df

def func_correct_name(df, name_col):
    list_of_name=['ZESTAW UPOMIN.']
    list_of_correct_name=['ZESTAWUPOMIN.']
    for zla_nazwa, dobra_nazwa in zip(list_of_name, list_of_correct_name):
        df[name_col]=df[name_col].replace(zla_nazwa, dobra_nazwa, regex=False)
    return df

In [203]:

    
def func_filtr_feler2(df, name_col):
    df['czy_z_f']=df[name_col].apply(lambda x: 'tak' if x[:2]=='F/' else 'nie')
    df['name_tmp']=df[name_col].apply(lambda x: x[2:] if x[:2]=='F/' else x)
    df_tmp_f=df.loc[df['czy_z_f']=='tak']
    df=df.loc[df['czy_z_f']=='nie']
    display(len(df_tmp_f))
    # display(df_tmp_f.loc[df_tmp_f['name_tmp']=='ZESTAWUPOMIN.'])
    # display(df.loc[df['name_tmp']=='ZESTAWUPOMIN.'])
    df['czy_jest_taki_felere']='nie'
    df_tmp_f['czy_dodany']='nie'
    df_tmp_f['NOWA_NAZWA']=''
    df_tmp_f['NOWA_RODZAJ']=''
    df_tmp_f['NOWA_KOD_TEMA']=''
    for name, rodzaj, ind in zip(df_tmp_f['name_tmp'], df_tmp_f['RODZAJ1'], df_tmp_f.index):
        if df.loc[df[name_col].str.contains(name)].loc[df['RODZAJ1']==rodzaj].empty:
            continue
        else:
            df_tmp_f.loc[ind, 'NOWA_NAZWA']=df.loc[df.loc[df[name_col].str.contains(name)].loc[df['RODZAJ1']==rodzaj].index][name_col].values[0]
            df_tmp_f.loc[ind, 'NOWA_RODZAJ']=df.loc[df.loc[df[name_col].str.contains(name)].loc[df['RODZAJ1']==rodzaj].index]['RODZAJ1'].values[0]
            df_tmp_f.loc[ind, 'NOWA_KOD_TEMA']=df.loc[df.loc[df[name_col].str.contains(name)].loc[df['RODZAJ1']==rodzaj].index]['KOD_TEMA'].values[0]
            # df.loc[df.loc[df[name_col].str.contains(name)].loc[df['RODZAJ1']==rodzaj].index, 'czy_jest_taki_felere']='tak'
            # df_tmp_f.loc[ind, 'czy_dodany']='tak'
     
    df=df.loc[df['czy_jest_taki_felere']=='tak']
    display(len(df))
    df=df.append(df_tmp_f.loc[df_tmp_f['czy_dodany']=='nie'], ignore_index=True)
    return df_tmp_f.reset_index().drop('index', axis=1)

In [128]:

def func_filtr_feler(df, name_col):
    df['name_tmp']=df[name_col].apply(lambda x: x[2:] if x[:2]=='F/' else x)
    df['czy_z_f']=df[name_col].apply(lambda x: 'tak' if x[:2]=='F/' else 'nie')
    
    df_tmp=df.groupby(['FAKTURA','name_tmp']).agg({name_col:'nunique'}).reset_index()
    df_tmp=df_tmp.loc[df_tmp[name_col]==2].drop(name_col, axis=1)
    df=df.merge(df_tmp, how='inner', on=['FAKTURA','name_tmp'])
    df=df.loc[df['czy_z_f']=='tak'].reset_index().drop('index', axis=1)
    return df

In [156]:
df_test=pd.read_excel('raport_felery/FELERY2020_05_2021.xlsx')
df_test=df_test.loc[df_test['ZNACZNIK']==19]
df_test['name_with_f']=df_test['NAZWA'].apply(lambda x: x if x[:2]=='F/' else 'F/'+x)
df_test['name_without_f']=df_test['NAZWA'].apply(lambda x: x[2:] if x[:2]=='F/' else x)


In [106]:
df=func_read_data('raport_felery/test_f56')
df['CENA_Z1']=func_str_to_float(df['CENA_Z1'])
df['CENA_Z2']=func_str_to_float(df['CENA_Z2'])
df['ILOSC1']=func_str_to_float(df['ILOSC1'])
df['ILOSC2']=func_str_to_float(df['ILOSC2'])
df['CENA_WAL']=func_str_to_float(df['CENA_WAL'])
df['CENA']=func_str_to_float(df['CENA'])
df=func_add_time(df)
df=func_correct_name(df, 'NAZWA1')
df=df.loc[df['ZNACZNIK']==19]

In [204]:
_=func_filtr_feler2(df, 'NAZWA1')

35

0

In [201]:
#_.merge(df_test[['Z_DNIA','name_without_f','RODZAJ','ILOSC','NAZWA', 'ZNACZNIK']], how='outer', left_on=['Z_DNIA','NAZWA1','RODZAJ1', 'ZNACZNIK'], right_on=['Z_DNIA','name_without_f','RODZAJ', 'ZNACZNIK'])

In [205]:
_

,FAKTURA,Z_DNIA,NAZWA1,RODZAJ1,CENA_Z1,ILOSC1,NAZWA2,RODZAJ2,CENA_Z2,ILOSC2,KOD_TEMA,VAT_P,CENA,ZNACZNIK,TRZY,NAZWA_IMP,CENA_WAL,year,month,name_tmp,czy_z_f,name_len,czy_dodany,NOWA_NAZWA,NOWA_RODZAJ,NOWA_KOD_TEMA
0,SC5600021985,2020-01-08,F/K/GINA/01,43X31X18 B K5,23.42,0.0,F/K/GINA/01,43X31X18 B K5,23.42,2.0,321422,23,0.0,19,K5,43X31X18 BASKET WHITE + BLUE LACE,5.040,2020,1,K/GINA/01,tak,11,nie,K/GINA/01,43X31X18 B K5,73502
1,SC5600022713,2020-02-27,F/GNIOTEK,8CM 134,1.23,0.0,F/GNIOTEK,8CM 134,1.23,23.0,355911,23,0.0,19,134,FACE BALL,0.295,2020,2,GNIOTEK,tak,9,nie,GNIOTEK,8CM 134,93624
2,SC5600023066,2020-03-11,F/K/TINA/01,49X34X28X13X38 K7,21.66,0.0,F/K/TINA/01,49X34X28X13X38 K7,21.66,1.0,367072,23,0.0,19,K7,WOOD & WILLOW,4.080,2020,3,K/TINA/01,tak,11,nie,K/TINA/01,49X34X28X13X38 K7,73519
3,SC5600023353,2020-04-14,F/K/RENE/03,34X22X9 K7,10.40,0.0,F/K/RENE/03,34X22X9 K7,10.40,1.0,325647,23,0.0,19,K7,WILLOW,1.960,2020,4,K/RENE/03,tak,11,nie,K/RENE/03,34X22X9 K7,73515
4,SC5600025655,2020-10-08,F/K/GLORIA/02,37X25X31 B+NIEB,18.54,0.0,F/K/GLORIA/02,37X25X31 B+NIEB,18.54,1.0,321704,23,0.0,19,K5,37X25X31 BASKET WHITE+BLUE,3.990,2020,10,K/GLORIA/02,tak,13,nie,K/GLORIA/02,37X25X31 B+NIEB,73506
5,SC5600025655,2020-10-08,F/K/KOR/INA/2/B,18X16X7 SERCE,5.96,0.0,F/K/KOR/INA/2/B,18X16X7 SERCE,5.96,1.0,227015,23,0.0,19,388,18X16X7H HEART,1.360,2020,10,K/KOR/INA/2/B,tak,15,nie,K/KOR/INA/2/B,18X16X7 SERCE,40117
6,SC5600025655,2020-10-08,F/K/KOR/INA/3/B,14X12X7 SERCE,5.09,0.0,F/K/KOR/INA/3/B,14X12X7 SERCE,5.09,1.0,227016,23,0.0,19,388,13.5X12X7H HEART,1.160,2020,10,K/KOR/INA/3/B,tak,15,nie,K/KOR/INA/3/B,14X12X7 SERCE,40118
7,SC5600025655,2020-10-08,F/K/KOR/IZA/1/B,23X23X9 B,9.00,0.0,F/K/KOR/IZA/1/B,23X23X9 B,9.00,1.0,379202,23,0.0,19,388,WHITE/1 23X23X9,1.760,2020,10,K/KOR/IZA/1/B,tak,15,nie,K/KOR/IZA/1/B,23X23X9 B,17128
8,SC5600025655,2020-10-08,F/K/KOR/MOLY/1/B,21RDX8,6.65,0.0,F/K/KOR/MOLY/1/B,21RDX8,6.65,1.0,227017,23,0.0,19,388,WHITE 23.5X8H ROUND,1.300,2020,10,K/KOR/MOLY/1/B,tak,16,nie,K/KOR/MOLY/1/B,21RDX8,40126
9,SC5600025655,2020-10-08,F/K/KOR/MOLY/2/B,24RDX8,8.69,0.0,F/K/KOR/MOLY/2/B,24RDX8,8.69,1.0,336081,23,0.0,19,388,23.5X8H ROUND,1.700,2020,10,K/KOR/MOLY/2/B,tak,16,nie,K/KOR/MOLY/2/B,24RDX8,40125


In [182]:
df_slownik=pd.read_csv('../słowniki/slownik.csv',
                       encoding='Cp1250',
                      sep="|")
df_slownik=df_slownik.loc[df_slownik['IS_DELETED']=='N'][['KOD_TEMA', 'NAZWA','RODZAJ','ZNACZNIK', 'TRZY', 'NAZWA_IMP', 'CENA_WAL', 'SYMBOL_IMP']]


In [183]:
df_slownik

,KOD_TEMA,NAZWA,RODZAJ,ZNACZNIK,TRZY,NAZWA_IMP,CENA_WAL,SYMBOL_IMP
0,14933,CHP/G/GILBIRT,45X45/X2,7,49,,"2,6",GILBIRT
1,141407,C/ALMA4/WAZ/1,27X10X25 B,5,279,WHITE SHINY GLOBAL 25CM FRILL,"5,16",K0495025F WHITE
2,141408,C/ALORA/MIS/1/B,23X23X15,5,154,"9"" VEIN CARVING VASE","5,64",XG3994 WHITE
3,141409,C/BELLA/FIG/18/C,17X11X16 CZAR,5,243,17.5X11X16 BLACK,"3,38",HY12-201HE
4,141431,C/FIG/09C/SREB,12X12X19 DRZEWO,5,198,CERAMIC DECORATION 12X12X19 SILVER,"3,5",4473A SILVER
...,...,...,...,...,...,...,...,...
230097,416108,F/SI/R/DONA/05/S,70X140 (X3) 500,2,FTM,GREY 70X140 500GSM,"3,381",NEW RIKI
230098,408778,RE/ROANA/01/B,70X140 (X3) 540,2,MXX,ZERO TWIST 540GSM 70X140,"7,02",ROANA WHITE
230099,416748,F/60587/B,300/01,1,DEV,PLAIN FABRIC 300CM WITH LB,"3,6",DLN60587 01
230100,416884,CHO/NATU/9,85X85 NAT+BR,8,265,"NATURAL +BROWN PIPING, 8CM EDGE 85X85","1,98",YG5727


In [179]:
_

,FAKTURA,Z_DNIA,NAZWA1,RODZAJ1,CENA_Z1,ILOSC1,NAZWA2,RODZAJ2,CENA_Z2,ILOSC2,KOD_TEMA,VAT_P,CENA,ZNACZNIK,TRZY,NAZWA_IMP,CENA_WAL,year,month,name_tmp,czy_z_f,name_len
245,SC5600021985,2020-01-08,F/K/GINA/01,43X31X18 B K5,23.42,0.0,F/K/GINA/01,43X31X18 B K5,23.42,2.0,321422,23,0.0,19,K5,43X31X18 BASKET WHITE + BLUE LACE,5.040,2020,1,K/GINA/01,tak,11
2886,SC5600022713,2020-02-27,F/GNIOTEK,8CM 134,1.23,0.0,F/GNIOTEK,8CM 134,1.23,23.0,355911,23,0.0,19,134,FACE BALL,0.295,2020,2,GNIOTEK,tak,9
4224,SC5600023066,2020-03-11,F/K/TINA/01,49X34X28X13X38 K7,21.66,0.0,F/K/TINA/01,49X34X28X13X38 K7,21.66,1.0,367072,23,0.0,19,K7,WOOD & WILLOW,4.080,2020,3,K/TINA/01,tak,11
5456,SC5600023353,2020-04-14,F/K/RENE/03,34X22X9 K7,10.40,0.0,F/K/RENE/03,34X22X9 K7,10.40,1.0,325647,23,0.0,19,K7,WILLOW,1.960,2020,4,K/RENE/03,tak,11
12434,SC5600025655,2020-10-08,F/K/GLORIA/02,37X25X31 B+NIEB,18.54,0.0,F/K/GLORIA/02,37X25X31 B+NIEB,18.54,1.0,321704,23,0.0,19,K5,37X25X31 BASKET WHITE+BLUE,3.990,2020,10,K/GLORIA/02,tak,13
12435,SC5600025655,2020-10-08,F/K/KOR/INA/2/B,18X16X7 SERCE,5.96,0.0,F/K/KOR/INA/2/B,18X16X7 SERCE,5.96,1.0,227015,23,0.0,19,388,18X16X7H HEART,1.360,2020,10,K/KOR/INA/2/B,tak,15
12436,SC5600025655,2020-10-08,F/K/KOR/INA/3/B,14X12X7 SERCE,5.09,0.0,F/K/KOR/INA/3/B,14X12X7 SERCE,5.09,1.0,227016,23,0.0,19,388,13.5X12X7H HEART,1.160,2020,10,K/KOR/INA/3/B,tak,15
12437,SC5600025655,2020-10-08,F/K/KOR/IZA/1/B,23X23X9 B,9.00,0.0,F/K/KOR/IZA/1/B,23X23X9 B,9.00,1.0,379202,23,0.0,19,388,WHITE/1 23X23X9,1.760,2020,10,K/KOR/IZA/1/B,tak,15
12438,SC5600025655,2020-10-08,F/K/KOR/MOLY/1/B,21RDX8,6.65,0.0,F/K/KOR/MOLY/1/B,21RDX8,6.65,1.0,227017,23,0.0,19,388,WHITE 23.5X8H ROUND,1.300,2020,10,K/KOR/MOLY/1/B,tak,16
12439,SC5600025655,2020-10-08,F/K/KOR/MOLY/2/B,24RDX8,8.69,0.0,F/K/KOR/MOLY/2/B,24RDX8,8.69,1.0,336081,23,0.0,19,388,23.5X8H ROUND,1.700,2020,10,K/KOR/MOLY/2/B,tak,16
